### Loading Dataset

In [2]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(source("tree_term_climb.R"))
suppressPackageStartupMessages(library(beaveR))

saveDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts/environment/real_datasets/GSE100505_EDL_MAST_subsamp/mode=gcbias/post_type=gibbs_nreps=100_tf=100/DE"
load(file.path(saveDir, "tseCons.RData"))

In [3]:
treeCons <- rowTree(tseCons)
l <- length(treeCons$tip)

quantDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/real_datasets/GSE100505_EDL_MAST_subsamp/sal_out/mode_gcbias=True/posttype=gibbs_npost=100_tf=100"
metaData <- read.delim("/fs/cbcb-lab/rob/students/noor/Uncertainity/real_datasets/GSE100505_EDL_MAST/SRR_Acc_List.txt")
samples <- metaData$RUN
files <- file.path(quantDir, samples, "quant.sf")
colData <- cbind(data.frame(files = files, names = samples), condition = as.factor(metaData$TissueName))
seMuscle <- tximeta::tximeta(colData)

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


found matching transcriptome:
[ GENCODE - Mus musculus - release M25 ]

loading existing TxDb created: 2023-02-17 18:26:37

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

loading existing transcript ranges created: 2023-02-17 18:26:39

fetching genome info for GENCODE



In [4]:
y <- fishpond::scaleInfReps(seMuscle)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

In [5]:
yAll <- computeSizeFactors(tseCons)
yAll <- scaleInfReps(yAll)
yAll <- labelKeep(yAll)
set.seed(1)
yAll <- swish(yAll, x="condition")
sum(mcols(yAll)[["keep"]])

Setting inf rep depth to mean depth



Progress:   1 on 100Progress:   2 on 100Progress:   3 on 100Progress:   4 on 100Progress:   5 on 100Progress:   6 on 100Progress:   7 on 100Progress:   8 on 100Progress:   9 on 100Progress:  10 on 100Progress:  11 on 100Progress:  12 on 100Progress:  13 on 100Progress:  14 on 100Progress:  15 on 100Progress:  16 on 100Progress:  17 on 100Progress:  18 on 100Progress:  19 on 100Progress:  20 on 100Progress:  21 on 100Progress:  22 on 100Progress:  23 on 100Progress:  24 on 100Progress:  25 on 100Progress:  26 on 100Progress:  27 on 100Progress:  28 on 100Progress:  29 on 100Progress:  30 on 100Progress:  31 on 100Progress:  32 on 100Progress:  33 on 100Progress:  34 on 100Progress:  35 on 100Progress:  36 on 100Progress:  37 on 100Progress:  38 on 100Progress:  39 on 100Progress:  40 on 100Progress:  41 on 100Progress:  42 on 100Progress:  43 on 100Progress:  44 on 100Progress:  45 on 100Progress:  46 on 100Progress:  47 on 100Progress:  48 on 100Progress:  49 on 100Progress:  50 on 100

[1] 95860

#### Finding differentially expressed transcripts on the subsampled dataset

In [9]:
dTxpsSub <- lapply(c(0.01,0.05,0.1), function(x) rownames(y)[which(mcols(y)[["qvalue"]] <= x)])
sapply(dTxpsSub, length)

[1] 2753 5540 7696

In [10]:
mapDf <- rowData(seMuscle)
txpGenes <- lapply(dTxpsSub, function(txps) unique(unlist(mapDf[txps, "gene_id"])))##Genes that map to transcripts
sapply(txpGenes, length)

[1] 2398 4529 5961

#### Finding differentially expressed genes on the subsampled dataset

In [13]:
gse <- summarizeToGene(seMuscle)
yg <- fishpond::scaleInfReps(gse)
yg <- labelKeep(yg)
set.seed(1)
yg <- swish(yg, x="condition")

loading existing TxDb created: 2023-02-17 18:26:37

obtaining transcript-to-gene mapping from database



ERROR: Error: database or disk is full


In [9]:
dgesSub <- lapply(c(0.01,0.05,0.1), function(x) rownames(yg)[which(mcols(yg)[["qvalue"]] <= x)])
sapply(dgesSub, length)

[1] 1933 3553 4890

In [16]:
envMain <- "environment/real_datasets/GSE100505_EDL_MAST/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/DE"
load(file.path(envMain, "dTxps.RData"))
load(file.path(envMain, "dges.RData"))

#### TreeDE, with the total number of nodes, inner nodes and their height distribution

In [14]:
load(file.path(saveDir, "tAfterBHMoreClimbMIRV.RData"))
treeDESub <- tAfterBHMoreClimbMIRV[[1]]
sapply(treeDESub, length) ## number of differentially expressed nodes
sapply(treeDESub, function(nodes) sum(nodes > l)) ## number of inner nodes
lapply(treeDESub, function(nodes) table(node.depth(treeCons,2)[nodes])) ## Distribution of node heights

[1]  804 5269 7787

[1]  333 2018 2749

[[1]]

  1   2   3   4   5   6   7   8   9 
471 174  87  35  18  11   2   3   3 

[[2]]

   1    2    3    4    5    6    7    8    9   10   11   12 
3251 1087  531  195  106   54   22   12    7    1    2    1 

[[3]]

   1    2    3    4    5    6    7    8    9   10   11   12   15 
5038 1535  678  268  144   67   24   15   10    2    4    1    1 


In [17]:
### Intersection for transcripts and genes between subsampled and original dataset
commTxps <- sapply(seq_along(dTxps), function(i) intersect(dTxps[[i]], dTxpsSub[[i]]))
sapply(commTxps, length)
diffTxps <- sapply(seq_along(dTxps), function(i) setdiff(dTxps[[i]], dTxpsSub[[i]]))
sapply(diffTxps, length)
                   
commGenes <- sapply(seq_along(dges), function(i) intersect(dges[[i]], dgesSub[[i]]))
sapply(commGenes, length)
diffGenes <- sapply(seq_along(dges), function(i) setdiff(dges[[i]], dgesSub[[i]]))
sapply(diffGenes, length)

[1] 2486 4990 7007

[1]  898 1794 2634

[1]  949 3314 4580

[1]   55 1362 1629

In [18]:
commTxps <- sapply(seq_along(dTxps), function(i) intersect(dTxps[[i]], dTxpsSub[[i]]))
sapply(commTxps, length)
diffTxps <- sapply(seq_along(dTxps), function(i) setdiff(dTxps[[i]], dTxpsSub[[i]]))
sapply(diffTxps, length)

[1] 2486 4990 7007

[1]  898 1794 2634

#### Number of differential transcripts in the original data mapped by the treeDE on the subsampled data

In [41]:
txpsTreeDE <- lapply(treeDESub, function(nodes) treeCons$tip[unlist(Descendants(treeCons, nodes))])
sapply(txpsTreeDE, length)
mapply(function(x,y) sum(x %in% y), txpsTreeDE, dTxps)

[1]  1462  9115 12929

[1]  850 4687 7053

In [22]:
children <- Descendants(treeCons, l+1, "children")
length(intersect(treeCons$tip[children[children < l]], dTxps[[1]]))

[1] 463

We get less intersection between treeDE nodes and original txps compared to DE between txps and 

#### Genes mapping to the differentially expressed inner nodes for the subsampled dataset

In [20]:
genesInnerNodesSub <- lapply(treeDESub, function(nodes) unique(unlist(mapDf[treeCons$tip[unique(unlist(Descendants(treeCons, nodes)))], "gene_id"])))
sapply(genesInnerNodesSub, length) ## Number of genes mapping to inner nodes
                             
sapply(mapply(function(x,y) intersect(unlist(x), y), genesInnerNodesSub, dges), length)

[1]  752 4602 6444

[1]  603 3346 4730

In [ ]:
genesInnerNodesSub <- lapply(treeDESub, function(nodes) unique(unlist(mapDf[treeCons$tip[unique(unlist(Descendants(treeCons, nodes)))], "gene_id"])))
sapply(genesInnerNodesSub, length) ## Number of genes mapping to inner nodes

In [42]:
sum(mcols(y)[["keep"]])
sum(mcols(yAll)[["keep"]])

sum(mcols(y)[treeCons$tip,"pvalue"] < 0.1, na.rm=T)
sum(mcols(yAll)[treeCons$tip,"pvalue"] < 0.1, na.rm=T)

sum(mcols(y)[treeCons$tip,"pvalue"] < 0.01, na.rm=T)
sum(mcols(yAll)[treeCons$tip,"pvalue"] < 0.01, na.rm=T)

txps <- treeCons$tip[which(mcols(y)[treeCons$tip,"pvalue"] < 0.01)]

head(mcols(y)[txps,c("log10mean", "pvalue", "qvalue")])
head(mcols(yAll)[txps,c("log10mean", "pvalue", "qvalue")])

[1] 32333

[1] 95860

[1] 12368

[1] 10627

[1] 5800

[1] 4069

DataFrame with 6 rows and 3 columns
                     log10mean      pvalue      qvalue
                     <numeric>   <numeric>   <numeric>
ENSMUST00000057438.6   2.53752 2.35054e-05 0.000731473
ENSMUST00000027286.6   3.38736 4.76139e-03 0.033358613
ENSMUST00000192078.1   1.41808 7.69152e-03 0.046466741
ENSMUST00000027217.8   1.32043 4.16138e-03 0.030454957
ENSMUST00000061334.9   2.27079 1.27053e-03 0.013277311
ENSMUST00000114246.3   2.82043 6.33996e-03 0.040745379

DataFrame with 6 rows and 3 columns
                     log10mean      pvalue    qvalue
                     <numeric>   <numeric> <numeric>
ENSMUST00000057438.6   2.53817 0.000702587 0.0117192
ENSMUST00000027286.6   3.38761 0.021966305 0.0786204
ENSMUST00000192078.1   1.41817 0.019888170 0.0735326
ENSMUST00000027217.8   1.32216 0.011665137 0.0522215
ENSMUST00000061334.9   2.27115 0.004781243 0.0296002
ENSMUST00000114246.3   2.82119 0.016164093 0.0642036

#### Number of differentially expressed transcripts for both

In [7]:
sum(mcols(y)[treeCons$tip,"qvalue"] < 0.01, na.rm=T)
sum(mcols(y)[treeCons$tip,"qvalue"] < 0.05, na.rm=T)
sum(mcols(y)[treeCons$tip,"qvalue"] < 0.1, na.rm=T)

sum(mcols(yAll)[treeCons$tip,"qvalue"] < 0.01, na.rm=T)
sum(mcols(yAll)[treeCons$tip,"qvalue"] < 0.05, na.rm=T)
sum(mcols(yAll)[treeCons$tip,"qvalue"] < 0.1, na.rm=T)

[1] 2753

[1] 5540

[1] 7696

[1] 574

[1] 4253

[1] 6624

In [40]:
summary(abs(assays(yAll)[["infRep1"]][treeCons$tip,] - assays(y)[["infRep1"]][treeCons$tip,])/(assays(y)[["infRep1"]][treeCons$tip,]+1))
summary(abs(assays(yAll)[["infRep1"]][treeCons$tip,] - assays(y)[["infRep1"]][treeCons$tip,]))
sum(abs(assays(yAll)[["infRep1"]][treeCons$tip,] - assays(y)[["infRep1"]][treeCons$tip,]) > 100)

   SRR5758666         SRR5758667         SRR5758668          SRR5758669      
 Min.   :0.000000   Min.   :0.000000   Min.   :0.0000000   Min.   :0.000000  
 1st Qu.:0.004203   1st Qu.:0.000795   1st Qu.:0.0004425   1st Qu.:0.001296  
 Median :0.012433   Median :0.002128   Median :0.0007103   Median :0.002461  
 Mean   :0.020492   Mean   :0.007345   Mean   :0.0063597   Mean   :0.008242  
 3rd Qu.:0.015787   3rd Qu.:0.006040   3rd Qu.:0.0060081   3rd Qu.:0.007663  
 Max.   :0.451266   Max.   :0.422306   Max.   :0.4181227   Max.   :0.472239  
   SRR5758670          SRR5758671         SRR5758630         SRR5758631      
 Min.   :0.0000000   Min.   :0.000000   Min.   :0.000000   Min.   :0.000000  
 1st Qu.:0.0004418   1st Qu.:0.001007   1st Qu.:0.001183   1st Qu.:0.001259  
 Median :0.0006874   Median :0.002151   Median :0.002622   Median :0.002825  
 Mean   :0.0070543   Mean   :0.009488   Mean   :0.009055   Mean   :0.009354  
 3rd Qu.:0.0067835   3rd Qu.:0.008163   3rd Qu.:0.007672   3rd Q

   SRR5758666          SRR5758667         SRR5758668         SRR5758669       
 Min.   :    0.000   Min.   :   0.000   Min.   :  0.0000   Min.   :   0.0000  
 1st Qu.:    0.007   1st Qu.:   0.002   1st Qu.:  0.0011   1st Qu.:   0.0032  
 Median :    0.051   Median :   0.017   Median :  0.0127   Median :   0.0249  
 Mean   :    2.697   Mean   :   0.460   Mean   :  0.2110   Mean   :   0.4226  
 3rd Qu.:    0.387   3rd Qu.:   0.106   3rd Qu.:  0.0764   3rd Qu.:   0.1469  
 Max.   :23472.319   Max.   :3523.453   Max.   :885.7109   Max.   :1930.9262  
   SRR5758670         SRR5758671          SRR5758630          SRR5758631       
 Min.   :  0.0000   Min.   :   0.0000   Min.   :   0.0000   Min.   :   0.0000  
 1st Qu.:  0.0012   1st Qu.:   0.0023   1st Qu.:   0.0025   1st Qu.:   0.0027  
 Median :  0.0140   Median :   0.0232   Median :   0.0219   Median :   0.0221  
 Mean   :  0.1964   Mean   :   0.4080   Mean   :   0.5110   Mean   :   0.5351  
 3rd Qu.:  0.0870   3rd Qu.:   0.1373   3rd Qu.

[1] 496

In [36]:
sum((assays(y)[["infRep1"]][treeCons$tip,])==0)

[1] 144097